In [1]:
!pip install transformers
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git

In [2]:
## Merging base model and fine-tuned params
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel
# import torch

# def merge_models(base_model_name_or_path, peft_model_path, output_dir, device="auto", push_to_hub=False):
#     if device == 'auto':
#         device_arg = {'device_map': 'auto'}
#     else:
#         device_arg = {'device_map': {"": device}}

#     print(f"Loading base model: {base_model_name_or_path}")
#     base_model = AutoModelForCausalLM.from_pretrained(
#         base_model_name_or_path,
#         return_dict=True,
#         torch_dtype=torch.float16,
#         **device_arg
#     )

#     print(f"Loading PEFT: {peft_model_path}")
#     model = PeftModel.from_pretrained(base_model, peft_model_path, **device_arg)
#     print(f"Running merge_and_unload")
#     model = model.merge_and_unload()

#     tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)

#     if push_to_hub:
#         print(f"Saving to hub ...")
#         model.push_to_hub(f"{output_dir}", use_temp_dir=False)
#         tokenizer.push_to_hub(f"{output_dir}", use_temp_dir=False)
#     else:
#         model.save_pretrained(f"{output_dir}")
#         tokenizer.save_pretrained(f"{output_dir}")
#         print(f"Model saved to {output_dir}")

In [3]:
# base_model_name_or_path = "lmsys/vicuna-7b-v1.3" # Path to the original base model
# peft_model_path = "/kaggle/input/vicuna-7b-fine-tuned" # Path to the directory containing the fine-tuned model
# output_dir = "/kaggle/working/merged_model/" # Path to the desired output directory

# merge_models(base_model_name_or_path, peft_model_path, output_dir)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/kaggle/input/vicuna-7b-fine-tuned-merged-model"
tokenizer_path = "/kaggle/input/vicuna-7b-fine-tuned-merged-model"

model = AutoModelForCausalLM.from_pretrained(model_path, load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def summarize(paragraph):
    # Tokenize the input paragraph
    input_tokens = tokenizer.encode(paragraph, return_tensors='pt', truncation=True)
    
    # Generate summary using the model
    summary_ids = model.generate(input_tokens, max_new_tokens=256)
    
    # Decode the summary and return
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [6]:
paragraph = """In the tranquil town of Willowbrook, nestled between emerald hills and sparkling rivers, there resided a curious inventor named Oliver. Oliver's workshop was a haven of creativity, filled with gears, cogs, blueprints, and mysterious gadgets that whirred and buzzed at all hours. His mind was a whirlpool of ideas, constantly churning out innovative concepts that were far ahead of his time.

Oliver's latest invention was a mechanical bird, designed with intricate detail to mimic the flight and song of a real bird. Crafted from brass and adorned with delicate feathers, it was a marvel of engineering. Oliver spent countless nights fine-tuning the wings, programming the melody, and ensuring that every movement was graceful and lifelike.

The townsfolk of Willowbrook were often skeptical of Oliver's eccentric ways, but they couldn't deny the charm of his mechanical bird. Children would gather outside his window, eyes wide with wonder, as the bird took flight, circling the room and filling the air with a sweet, melodious tune. It was not just a machine; it was a piece of art, a symbol of human ingenuity and the endless possibilities of the mind.

Oliver's inventions were not driven by fame or fortune but by an insatiable curiosity and a desire to push the boundaries of what was possible. He believed that technology could be harnessed to create beauty, to inspire, and to bring joy to people's lives. His mechanical bird was a testament to that belief.

But Oliver's creativity was not without challenges. The path to innovation was fraught with obstacles, failures, and doubts. There were moments when he questioned his abilities, when the solutions seemed elusive, and the problems insurmountable. Yet, he persevered, driven by a relentless passion and a conviction that he was on the brink of something extraordinary.

As the years went by, Oliver's reputation grew, and his workshop became a place of pilgrimage for aspiring inventors and thinkers. They came to learn, to be inspired, and to witness the magic that was born from the fusion of art and science. Oliver's mechanical bird was no longer just a local curiosity; it was a symbol of human potential, a reminder that creativity knows no bounds, and that the pursuit of excellence is a journey worth taking.

In the end, Oliver's legacy was not defined by the inventions he created but by the lives he touched, the minds he ignited, and the sense of wonder he instilled in those who were fortunate enough to cross his path. His mechanical bird, perched on a shelf in his workshop, continued to sing its song, a timeless melody that resonated with the dreamers, the doers, and the believers in the extraordinary power of human creativity."""

In [7]:
    prompt = (
        f"Below is an instruction that describes a task. Write a response that appropriately completes the request. ###\n"
        f"Instruction: You are tasked to write a summarisation of a body of text. You must only reply with the summarisation of the following text: <text>{paragraph}<text> ###\n"
        f"Response: "
    )

In [8]:
summary = summarize(prompt)
print("Original Paragraph:", paragraph)
print("Summary:", summary)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Original Paragraph: In the tranquil town of Willowbrook, nestled between emerald hills and sparkling rivers, there resided a curious inventor named Oliver. Oliver's workshop was a haven of creativity, filled with gears, cogs, blueprints, and mysterious gadgets that whirred and buzzed at all hours. His mind was a whirlpool of ideas, constantly churning out innovative concepts that were far ahead of his time.

Oliver's latest invention was a mechanical bird, designed with intricate detail to mimic the flight and song of a real bird. Crafted from brass and adorned with delicate feathers, it was a marvel of engineering. Oliver spent countless nights fine-tuning the wings, programming the melody, and ensuring that every movement was graceful and lifelike.

The townsfolk of Willowbrook were often skeptical of Oliver's eccentric ways, but they couldn't deny the charm of his mechanical bird. Children would gather outside his window, eyes wide with wonder, as the bird took flight, circling the 